In [13]:
using Pkg
Pkg.add("IJulia")
Pkg.add("ITensors")
Pkg.add("ITensorMPS")
Pkg.add("LinearAlgebra")
Pkg.add("Random")
Pkg.add("Plots")
Pkg.add("Printf")
Pkg.add("OrderedCollections")

using ITensors, ITensorMPS, LinearAlgebra, OrderedCollections, HDF5

   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Leonardo.Borello\.julia\environments\v1.11\Manifest.toml`


In [14]:
using ITensors, ITensorMPS, LinearAlgebra


    N       = 10
    sites = siteinds("S=1/2", N)

    
    Jx      = 5.0         
    Jy      = 6.0         
    Jz      = 5.0         
    h       = 0.5       
    nsweeps = 10
    cutoff  = [1e-10]
    maxdim = [100]
    
    os = OpSum()
    for j in 1:N-1
        os += Jx, "Sx", j, "Sx", j+1
        os += Jy, "Sy", j, "Sy", j+1
        os += Jz, "Sz", j, "Sz", j+1
    end
    for j in 1:N
        os += -h, "Sx", j
    end
    H = MPO(os, sites)

    
    psi0 = random_mps(sites, n -> "Dn")
    energy, psi1 = dmrg(H, psi0; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff)

  

    psi_non_truncated = orthogonalize!(psi1, 1)
    product_state = MPS(sites, "Up")

After sweep 1 energy=-22.416182761116175  maxlinkdim=4 maxerr=9.99E-16 time=0.007
After sweep 2 energy=-22.741987160836967  maxlinkdim=16 maxerr=9.96E-11 time=0.009
After sweep 3 energy=-22.74204141163551  maxlinkdim=21 maxerr=8.71E-11 time=0.024
After sweep 4 energy=-22.74204141133242  maxlinkdim=20 maxerr=9.88E-11 time=0.018
After sweep 5 energy=-22.74204141133244  maxlinkdim=20 maxerr=9.83E-11 time=0.017
After sweep 6 energy=-22.742041411332398  maxlinkdim=20 maxerr=9.83E-11 time=0.013
After sweep 7 energy=-22.74204141133238  maxlinkdim=20 maxerr=9.83E-11 time=0.016
After sweep 8 energy=-22.7420414113324  maxlinkdim=20 maxerr=9.83E-11 time=0.013
After sweep 9 energy=-22.742041411332398  maxlinkdim=20 maxerr=9.83E-11 time=0.011
After sweep 10 energy=-22.7420414113324  maxlinkdim=20 maxerr=9.83E-11 time=0.015


MPS
[1] ((dim=2|id=777|"S=1/2,Site,n=1"), (dim=1|id=894|"Link,l=1"))
[2] ((dim=1|id=894|"Link,l=1"), (dim=2|id=195|"S=1/2,Site,n=2"), (dim=1|id=284|"Link,l=2"))
[3] ((dim=1|id=284|"Link,l=2"), (dim=2|id=720|"S=1/2,Site,n=3"), (dim=1|id=517|"Link,l=3"))
[4] ((dim=1|id=517|"Link,l=3"), (dim=2|id=28|"S=1/2,Site,n=4"), (dim=1|id=171|"Link,l=4"))
[5] ((dim=1|id=171|"Link,l=4"), (dim=2|id=666|"S=1/2,Site,n=5"), (dim=1|id=697|"Link,l=5"))
[6] ((dim=1|id=697|"Link,l=5"), (dim=2|id=879|"S=1/2,Site,n=6"), (dim=1|id=0|"Link,l=6"))
[7] ((dim=1|id=0|"Link,l=6"), (dim=2|id=326|"S=1/2,Site,n=7"), (dim=1|id=804|"Link,l=7"))
[8] ((dim=1|id=804|"Link,l=7"), (dim=2|id=856|"S=1/2,Site,n=8"), (dim=1|id=464|"Link,l=8"))
[9] ((dim=1|id=464|"Link,l=8"), (dim=2|id=94|"S=1/2,Site,n=9"), (dim=1|id=336|"Link,l=9"))
[10] ((dim=1|id=336|"Link,l=9"), (dim=2|id=37|"S=1/2,Site,n=10"))


In [15]:
function generate_mpds(psi)
    N = length(psi)
    psi = copy(psi)
    mpds = Dict{Int, ITensor}()
    
    for n = 1
        i = inds(psi[n], "Site")[1]
        j = commoninds(psi[n], psi[n+1])[1]
        
        Array_tensor1 = Array(psi[n], i,j)

        T = zeros(2,2,2,2)
        T[:,:,1,1] = Array_tensor1

        i2,j2,k2,l2 = prime(siteinds(psi, n)), prime(siteinds(psi, n+1)), siteinds(psi, n), siteinds(psi, n+1)
        tensor = ITensor(T, i2,j2,k2,l2)
        U,S,V = svd(tensor, i2,j2)

        S1 = delta(commonind(U,S), commonind(S,V))
        
        mpds[n] = U * S1 * V  
    end

    for n in 2:N-1
        i = inds(psi[n], "Site")[1]
        j = commoninds(psi[n], psi[n+1])[1]
        k = commoninds(psi[n-1], psi[n])[1]

        Array_tensor2 = Array(psi[n], i,j,k)

        G = zeros(2,2,2,2)
        G[:,:,:,1] = Array_tensor2 

        i2,j2,k2,l2 = prime(siteinds(psi, n)), prime(siteinds(psi, n+1)), siteinds(psi, n), siteinds(psi, n+1)
        tensor = ITensor(G, i2,j2,k2,l2)

        U,S,V = svd(tensor, i2,j2)
        S1 = delta(commonind(U,S), commonind(S,V))
        
        mpds[n] = U * S1 * V
    end

    for n = N
        i = inds((psi[n]), "Site")[1]
        j = commoninds(psi[n-1], psi[n])[1]
        mpds[n] = replaceinds(psi[n], i => prime(i), j => inds((psi[n]), "Site")[1])
    end
    return mpds
end

#This function helps with turning U into dag(U), and reverses the order such that we can apply to the MPS and disentangle it

function dagger!(mpds)
    mpds_dag = [dag(t) for t in mpds]
    for t in mpds_dag
        swapprime!(t, 0=>1)
    end
    return reverse(mpds_dag)
end

# Generate a circuit of depth D, with corresponding MPD gates.

function deep!(psi_non_truncated, D)
    N = length(sites)
    circuit_layers = Vector{Vector{ITensor}}(undef, D)
    
    psi_current = copy(psi_non_truncated)
    for d in 1:D
        psi_trunc = truncate(psi_current, maxdim=2)
        mpds = generate_mpds(psi_trunc)
        ordered_keys = sort(collect(keys(mpds)))
        mpds_vec = [mpds[n] for n in ordered_keys]
        mpds_vec = dagger!(mpds_vec)

        
        circuit_layers[d] = mpds_vec
        # Apply the MPDs to psi_current to get the next state
        psi_current = apply(mpds_vec, psi_current)
    end
    
    return circuit_layers
end


function reverse_deep!(circuit_layers::Vector{Vector{ITensor}})
    reversed_circuit = Vector{Vector{ITensor}}(undef, length(circuit_layers))
    
    for (i, layer) in enumerate(reverse(circuit_layers))
        reversed_layer = Vector{ITensor}(undef, length(layer))
        for (j, tensor) in enumerate(reverse(layer))
            reversed_layer[j] = swapprime(dag(tensor), 0=>1)
        end
        reversed_circuit[i] = reversed_layer
    end
    
    return reversed_circuit
end

function attract!(reversed_circuit, product_state)
    D = length(reversed_circuit)
    state = copy(product_state)
    
    for d in 1:D
        state = apply(reversed_circuit[d], state)
    end

    return state
end

#Usage
#psi_non_truncated = copy(psi_non_truncated)
#product_state = copy(product_state )
#D = 1
#circuit_layers = deep!(psi_non_truncated, D)
#actual_circuit = reverse_deep!(circuit_layers)
#state = attract!(actual_circuit, product_state)
#fidelity = abs(inner(psi_non_truncated, state))

attract! (generic function with 1 method)

### Single Depth

In [39]:
#Usage
psi_non_truncated = copy(psi_non_truncated)
product_state = copy(product_state)
D = 1
circuit_layers = deep!(psi_non_truncated, D)
actual_circuit = reverse_deep!(circuit_layers)
state = attract!(actual_circuit, product_state)
norm = abs(inner(psi_non_truncated, state))


using HDF5
function package!(actual_circuit)
    # actual_circuit is an array of layers, each layer being an array of gates.
    D = length(actual_circuit)
    packaged_layers = Vector{Any}(undef, D)
    
    for d in 1:D
        layer = actual_circuit[d]
        N = length(layer)
        matrix_gates = Vector{Any}(undef, N)
        
        # Process the first N-1 gates as two-qubit gates.
        for n in 1:N-1
            gate = layer[n]
            s1_out = inds(gate, "Site")[1]
            s2_out = inds(gate, "Site")[2]
            s1_in  = inds(gate, "Site")[3]
            s2_in  = inds(gate, "Site")[4]
            gate_array = Array(gate, (s1_out, s2_out, s1_in, s2_in))
            matrix_gates[n] = reshape(gate_array, (4,4))
        end
        
        # Process the final gate (assumed to be a one-qubit gate).
        gate = layer[N]
        s1_out = inds(gate, "Site")[1]
        s1_in  = inds(gate, "Site")[2]
        gate_array = Array(gate, (s1_out, s1_in))
        matrix_gates[N] = reshape(gate_array, (2,2))
        
        packaged_layers[d] = matrix_gates
    end
    
    return packaged_layers
end

function savepackage!(filename::String, packaged_layers::Vector{Any})
    # Open an HDF5 file in write mode.
    h5open(filename, "w") do file
        # Loop over layers.
        for (layer_idx, layer) in enumerate(packaged_layers)
            # Loop over the gates in each layer.
            for (gate_idx, gate) in enumerate(layer)
                key = "layer$(layer_idx)_gate$(gate_idx)"
                write(file, key, gate)
            end
        end
    end
end

# Example usage:
# Suppose `actual_circuit` is already defined and you obtain packaged_layers:
packaged_layers = package!(actual_circuit)

# Let N be the number of sites (you can get it from the length of a layer) and D the depth.
N = length(actual_circuit[1])  # Number of gates in one layer corresponds to N sites.
D = length(actual_circuit)     # Depth of the circuit.

# Save the gates to an HDF5 file with a filename that encodes N and D.
filename = "gates_N$(N)_D$(D).h5"
savepackage!(filename, packaged_layers)
println("Saved circuit for N = $N and depth D = $D to ", filename)

Saved circuit for N = 10 and depth D = 1 to gates_N10_D1.h5


### Multiple depths

In [17]:
using HDF5

using OrderedCollections

ψ    = copy(psi_non_truncated)
prod = copy(product_state)

function generate_circuits_range(ψ0, prod0; depths=1:20)
    depths_vec = sort(collect(depths))           
    circuits = OrderedDict{Int, Vector{Any}}()
    for d in depths_vec
        ψ    = copy(psi_non_truncated)
        prod = copy(product_state)
        layers = deep!(ψ, d)                     
        circuits[d] = reverse_deep!(layers)      
    end
    return circuits
end

function package!(actual_circuit::Vector{<:Any})
    D = length(actual_circuit)
    packaged = Vector{Any}(undef, D)
    for d in 1:D
        layer = actual_circuit[d]
        N = length(layer)
        mats = Vector{Any}(undef, N)
        # two-qubit gates
        for n in 1:N-1
            gate = layer[n]
            o1,o2,i1,i2 = inds(gate, "Site")
            A = Array(gate, (o1,o2,i1,i2))
            mats[n] = reshape(A, 4, 4)
        end
        # single-qubit gate
        gate = layer[N]
        o,i = inds(gate, "Site")
        A = Array(gate, (o,i))
        mats[N] = reshape(A, 2, 2)

        packaged[d] = mats
    end
    return packaged
end

function savepackage!(filename::String, packaged_layers::Vector{Any})
    h5open(filename, "w") do file
        for (li, layer) in enumerate(packaged_layers)
            for (gi, gate_mat) in enumerate(layer)
                ds = "layer$(li)_gate$(gi)"
                write(file, ds, gate_mat)
            end
        end
    end
end


function save_all_depths!(
        ψ0, prod0;
        depths = 1:10,
        out_prefix = "gates_N" )
    
    circuits_by_depth = generate_circuits_range(ψ0, prod0; depths=depths)

   
    for (d, circuit_layers) in circuits_by_depth
        
        mats = package!(circuit_layers)

        N = length(circuit_layers[1])

        fname = "$(out_prefix)$(N)_D$(d).h5"

        savepackage!(fname, mats)
        println("Saved depth $d (N=$N) → $fname")
    end
end

save_all_depths!(psi_non_truncated, product_state; depths=1:10)

Saved depth 1 (N=10) → gates_N10_D1.h5
Saved depth 2 (N=10) → gates_N10_D2.h5
Saved depth 3 (N=10) → gates_N10_D3.h5
Saved depth 4 (N=10) → gates_N10_D4.h5
Saved depth 5 (N=10) → gates_N10_D5.h5
Saved depth 6 (N=10) → gates_N10_D6.h5
Saved depth 7 (N=10) → gates_N10_D7.h5
Saved depth 8 (N=10) → gates_N10_D8.h5
Saved depth 9 (N=10) → gates_N10_D9.h5
Saved depth 10 (N=10) → gates_N10_D10.h5
